In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import psycopg2


2024-06-26 16:59:39.594515: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-26 16:59:39.646862: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-26 16:59:39.647689: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-26 16:59:40.367802: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('./books_data.csv')

In [3]:
df = df[['bookId', 'title', 'series','author', 'description', 'coverImg']]

In [4]:
COVER_IMG_PREFIX = 'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/'
df['coverImg'] = df['coverImg'].str.replace(COVER_IMG_PREFIX, '')

In [5]:
df['description'] = df['description'].str.lower()

In [6]:
df = df.loc[df[['title', 'series']].dropna().drop_duplicates().index]
df.to_csv('processed_books.csv', index='False')

In [7]:
longest_description = df[df['description'].str.len().max() == df['description'].str.len()]



In [8]:
longest_description
EMB_SIZE = 384

MAX_TOKENS = 512
model = SentenceTransformer('Mihaiii/gte-micro-v2')


def create_embeddings(df: pd.DataFrame, model: SentenceTransformer) -> pd.DataFrame:
    
    descriptions = df['description']
    titles = df['title']
    
    desc_length = descriptions.str.len()
    
   
    for i in range(len(df)):
        book_emb = np.zeros(EMB_SIZE)
        title_emb = np.zeros(EMB_SIZE)
        if desc_length.iloc[i] > MAX_TOKENS:
            context_start = 0
            context_end = MAX_TOKENS
            length_left = desc_length.iloc[i]
            number_of_embeddings = 0
            while length_left > 0:
                book_emb += model.encode(str(descriptions.iloc[i][context_start:context_end]))
                context_start = context_end
                context_end += MAX_TOKENS if (context_end + MAX_TOKENS) <= length_left else length_left
                length_left = length_left - (context_end-context_start)
                number_of_embeddings+=1
            
            book_emb /= number_of_embeddings
        else:
            book_emb += model.encode(str(descriptions.iloc[i]))
        
        title_emb += model.encode(str(titles.iloc[i]))
        yield title_emb, book_emb


/home/hubert/anaconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
embs = create_embeddings(df, model)

In [10]:
connection = psycopg2.connect(database="movie_recommender", user="postgres", password="admin", host="localhost", port=5432)
cursor = connection.cursor()

cursor.execute("""
            CREATE EXTENSION vector;
            """)
cursor.execute("""
                   CREATE TABLE books (
                    id BIGSERIAL PRIMARY KEY,
                   title VARCHAR(255) NOT NULL,
                   series VARCHAR(255) NOT NULL,
                   author VARCHAR(400) NOT NULL,
                   img_path VARCHAR(50) NOT NULL,
                   title_embedding vector(384) NOT NULL,
                   book_embedding vector(384) NOT NULL
                   );
                """)
    
for i in tqdm(range(len(df))):
    title_emb, book_emb = next(embs)
    my_doc = {
        "id": i,
        "title": df['title'].iloc[i], 
        "series": df['series'].iloc[i],
        "author" : df['author'].iloc[i],
        "img_path": df['coverImg'].iloc[i],
        "title_embedding": title_emb.tolist(),
        "book_embedding": book_emb.tolist()
    }
    query = """
        INSERT INTO books
        VALUES (%(id)s, %(title)s, %(series)s, %(author)s, %(img_path)s, %(title_embedding)s, %(book_embedding)s)
        """
    cursor.execute(query, my_doc)
        
connection.commit()

cursor.execute('SELECT * FROM books;')
    
for r in cursor.fetchall(): # test if data was added
    print(r)
    break
cursor.close()

test


100%|█████████████████████████████████████| 23410/23410 [11:29<00:00, 33.94it/s]


(0, 'The Hunger Games', 'The Hunger Games #1', 'Suzanne Collins', '1586722975l/2767052.jpg', '[-0.32216313,0.19108859,0.38078955,-0.11360715,0.4076026,0.22038415,0.20585386,-0.05241995,-0.09283504,-0.10813488,-0.052215617,-0.6870852,0.33360338,0.20904367,0.3248169,-0.13583085,0.12304898,0.04010867,-0.45727253,0.25122276,0.16771533,-0.25164798,0.088192396,-0.5149356,0.27162886,0.078102656,-0.342764,-0.25815934,-0.8009394,-1.38028,0.29901856,-0.62722445,0.27586317,-0.1338308,-0.41473302,-0.14446852,0.14757724,0.4046147,-0.47384185,0.11537125,0.30367702,0.23429665,0.16966419,-0.39585754,-0.3256103,-0.74661785,-0.8190416,-0.06027178,0.54702514,-0.29127243,0.23202348,0.10369752,-0.13523245,0.15913181,0.2522553,0.19655915,0.26689917,0.1081843,0.5004477,0.14805709,0.20143974,0.1435274,-1.9021991,0.76581925,0.7934303,0.3537086,-0.013185523,-0.18649279,0.037441533,0.078522556,-0.36732295,0.4036165,0.5562254,0.33736104,-0.1133979,-0.06001417,0.0017560493,-0.45758605,-0.17529328,0.27207172,-0.131